# Project
Commercial banks receive many credit card applications. Many of them are rejected for various reasons, such as high loan balances, low income levels, or too many inquiries on a person's credit report, for example. Manually analyzing these applications is tedious, error-prone, and time-consuming (and time is money!). Fortunately, this task can be automated with the power of machine learning, and virtually all commercial banks do this nowadays. In this workbook, we will create an automatic credit card approval predictor using machine learning techniques, just as real banks do.

### The Data

The data is a small subset of the Credit Approval dataset from the UCI Machine Learning Repository, which shows the credit card applications received by a bank. The last column of the dataset is the target value.

# Proyecto
Los bancos comerciales reciben muchas solicitudes de tarjetas de crédito. Muchos de ellos son rechazados por muchas razones, como saldos de préstamos elevados, niveles de ingresos bajos o demasiadas consultas sobre el informe crediticio de una persona, por ejemplo. Analizar manualmente estas aplicaciones es tedioso, propenso a errores y requiere mucho tiempo (¡y el tiempo es dinero!). Afortunadamente, esta tarea se puede automatizar con el poder del aprendizaje automático y prácticamente todos los bancos comerciales lo hacen hoy en día. En este libro de trabajo, se  creará un predictor automático de aprobación de tarjetas de crédito utilizando técnicas de aprendizaje automático, tal como lo hacen los bancos reales.

## Datos

Los datos se encuentran en un  pequeño subconjunto del conjunto de datos de Aprobación de tarjetas de crédito del Repositorio de aprendizaje automático de UCI que muestra las solicitudes de tarjetas de crédito que recibe un banco. **La última columna del conjunto de datos es el valor objetivo.** 

## The Object
Supervised learning techniques are used to automate the credit card approval process for banks.

Data is preprocessed and supervised learning techniques are applied to find the best model and parameters for the task. The ultimate goal is to find the accuracy score of the best model as a numerical variable, best_score. The target is an accuracy score of at least 0.75.

----------------------------------------------------------------------------------------------------------------------------------------

El Objetivo
Se utilizan  técnicas de aprendizaje supervisado para automatizar el proceso de aprobación de tarjetas de crédito para bancos.

Previamente  se procesan los datos y se aplican técnicas de aprendizaje supervisado para encontrar el mejor modelo y parámetros para el trabajo. 
El objetivo final es encontrar la puntuación de precisión del mejor modelo como una variable numérica, best_score. se apúnta a una puntuación de precisión de al menos 0,75.



### Strategy
1. Preprocess the data

2. Prepare the data for modeling

3. Train the model

4. Find the best scoring model

------------------------------------------------------------------------------------------------------------------------------------

### Estrategia

1. Preprocesar los datos

2. Preparar los datos para modelar

3. Entrenamiento  del modelo

4. Encontrar el mejor modelo de puntuación



In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()



,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


### Data Preprocessing
Technical Explanation:
It involves transforming raw data into a suitable and clean format for analysis and modeling.

### Preprocesamiento de Datos

Exlicacion Tecnica:
Consiste en transformar datos crudos en un formato adecuado y limpio para su análisis y modelado. 


In [2]:
# para conocer si hay valores faltantes debo buscar valores nulos que no son especificamente NAN en cada columna con unique()
# en este caso lei la respuesta la columna 0 tiene valores "?"

print(cc_apps[0].unique())
cc_apps_replaced = cc_apps.replace("?", np.NaN)
cc_app_imputed = cc_apps_replaced.copy()
print(cc_apps_replaced.isna().sum())
print(cc_apps_replaced.info())

['b' 'a' '?']
0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    object 
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB
None


In [3]:
#recorra las columnas del dataframe 
for col in cc_app_imputed.columns:
    #si la columna es un objeto
    if cc_app_imputed[col].dtype== "object":
        # impute el valor mas frecuente utilizando filna para encontrar NA 
        cc_app_imputed[col]= cc_app_imputed[col].fillna(cc_app_imputed[col].value_counts().index[0])
    else:
        cc_app_imputed[col] = cc_app_imputed[col].fillna(cc_app_imputed[col].mean())
#Ahora la columna 0 tiene el valor mas frecuente y la columna que contiene numeros tiene el promedio

In [4]:
print(cc_app_imputed.isna().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [5]:
cc_app_encoded= pd.get_dummies(cc_app_imputed, drop_first=True)

In [6]:
X = cc_app_encoded.iloc[:,:-1].values
y= cc_app_encoded.iloc[:,[-1]].values

### Model Training
Technical Explanation:

Logistic regression is a powerful technique for binary classification that allows predicting the probability of an instance belonging to a specific class based on the provided features.


### Entrenamiento del Modelo
Explicacion Tecnica:

La regresión logística es una técnica poderosa para clasificación binaria que permite predecir la probabilidad de una instancia pertenezca a una clase específica basándose en las características proporcionadas.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33,random_state=42)

# Mostrar las dimensiones de los conjuntos
print("Dimensiones del conjunto de entrenamiento (X_train):", X_train.shape)
print("Dimensiones del conjunto de prueba (X_test):", X_test.shape)
print("Dimensiones del conjunto de entrenamiento (y_train):", y_train.shape)
print("Dimensiones del conjunto de prueba (y_test):", y_test.shape)

Dimensiones del conjunto de entrenamiento (X_train): (462, 382)
Dimensiones del conjunto de prueba (X_test): (228, 382)
Dimensiones del conjunto de entrenamiento (y_train): (462, 1)
Dimensiones del conjunto de prueba (y_test): (228, 1)


In [8]:
scaler =StandardScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

# Mostrar las primeras filas del conjunto de entrenamiento escalado
print("Conjunto de entrenamiento escalado:")
print(rescaledX_train[:5])

# Mostrar las primeras filas del conjunto de prueba escalado
print("Conjunto de prueba escalado:")
print(rescaledX_test[:5])

Conjunto de entrenamiento escalado:
[[-0.45027582  0.82127658 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [-0.39786487  0.73765827  0.69733785 ...  1.13451695 -0.12403473
  -0.2818497 ]
 [-0.6599196  -0.60023466 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [-0.74692177 -0.67047404 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [ 0.10003911  0.9885132  -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]]
Conjunto de prueba escalado:
[[-0.6599196  -0.68385297 -0.09423485 ...  1.13451695 -0.12403473
  -0.2818497 ]
 [-0.13581015 -0.68385297 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [-0.97438528 -0.51661635 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [ 0.38829931 -0.34937973 -0.49002119 ... -0.8814324  -0.12403473
  -0.2818497 ]
 [-0.97438528 -0.68385297 -0.49002119 ... -0.8814324  -0.12403473
   3.54799032]]


In [9]:
logreg = LogisticRegression()
logreg.fit(rescaledX_train, y_train)
y_train_pred= logreg.predict(rescaledX_train)


c:\Users\dmcar\miniconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The confusion matrix is a tool that allows you to evaluate the performance of a classification model.


La matriz de confusión es una herramienta que te permite evaluar el rendimiento de un modelo de clasificación.

In [10]:
print(confusion_matrix( y_train, y_train_pred))

[[203   1]
 [  1 257]]


### First Conclusion:

The confusion matrix indicates that the logistic regression model is performing extremely well on the training set, with only 2 errors out of a total of 462 predictions.

The score of this model is higher than recommended. In this case, we assume it is incorrect and will look to adjust hyperparameters to find a model that returns the target score of around 75%.

### Primera Conclusion :

La matriz de confusión indica que el modelo de regresión logística está funcionando extremadamente bien en el conjunto de entrenamiento, con solo 2 errores en un total de 462 predicciones

El score de este modelo es mas alto de lo recomndable, en este caso asumimos que es incorrecto y buscaremos ajustar hiperparametros para dar con el modelo que devuelba el score objetivo de alrededor 75%

### Finding the Best Model
Hyperparameters

Technical Explanation:

What does GridSearchCV do?

GridSearchCV performs an exhaustive search over a specified set of hyperparameter values. It will test all possible combinations of hyperparameters and evaluate the model's performance for each combination using cross-validation. In the end, it tells you which combination of hyperparameters is the best based on the chosen evaluation metric.


### Encontrar el Mejor modelo 

Hiperparametros

Explicacion Tecnica:

¿Qué hace GridSearchCV?

GridSearchCV realiza una búsqueda exhaustiva sobre un conjunto especificado de valores de hiperparámetros. Probará todas las combinaciones posibles de los hiperparámetros y evaluará el rendimiento del modelo para cada combinación usando validación cruzada. Al final, te dice cuál es la mejor combinación de hiperparámetros basada en la métrica de evaluación que se haya elegido 



In [11]:
tol =[0.01,0.001,0.00001]
max_iter=[100,150,200]

In [12]:
param_grid =dict(tol=tol, max_iter=max_iter)
grid_model = GridSearchCV(estimator=logreg,param_grid=param_grid,cv=5)
grid_model_result = grid_model.fit(rescaledX_train, y_train.ravel())

In [13]:
best_train_score, best_train_params = grid_model_result.best_score_,grid_model_result.best_params_

print("Best: %f using %s" % (best_train_score,best_train_params))



Best: 0.818256 using {'max_iter': 100, 'tol': 0.01}


In [14]:
best_model = grid_model_result.best_estimator_
best_score = best_model.score(rescaledX_test, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

Accuracy of logistic regression classifier:  0.7982456140350878


## Final Conclusion:
A project is carried out using data preprocessing strategies to create an ML model based on logistic regression. The initial result is evaluated, and it is concluded that hyperparameters need to be adjusted to select the best possible fit. The final result aligns with the desired objective.

Going forward, it remains to test other models and adjustments to determine continuous improvement in the model's results.

## Conclucion Final:

Se realiza un proyecto utlizando estrategias de preprosesamiento de datos, para poder realizar un modelo de ML basado en regression logistica. El primer resultado es evaludao y se concluye que se debe ajustar hipermparametros para seleccionar el mejor ajuste posible, El resultado final consuerda con el objetivo buscado.

En adelante resta probar otros modelos y otros ajustes para determinar una mejora continua en los resultados del modelo.

### Fuentes:

* https://projects.datacamp.com/projects/1908

* https://www.datacamp.com/tutorial/preprocessing-in-data-science-part-1-centering-scaling-and-knn

* https://www.datacamp.com/tutorial/preprocessing-in-data-science-part-2-centering-scaling-and-logistic-regression

* https://www.datacamp.com/tutorial/preprocessing-in-data-science-part-3-scaling-synthesized-data

* https://www.datacamp.com/tutorial/categorical-data